# Aula 4 - Técnicas de Engenharia de Atributos

Falamos bastante até o momento sobre a aplicação de técnicas para reduzir a dimensionalidade co conjunto de dados para evitar a queda de generalização dos modelos treinados. No entanto, é frequente encontrarmos situações onde existe a necessidade de *criar mais atributos* ao invés de eliminá-los. Isso é muito comum em conjuntos de dados em que as funções podem ser difíceis de serem estabelecidas. Veremos também que essa criação de atributos é comumente aplicada na etapa de análise exploratória dos dados.

O nome oficial dado a esse processo é *Engenharia de Atributos* - (literatura: *Feature Engineering*). Dentre as formas de se melhorar o desempenho de modelos, diria que a engenharia de atributos está entre as mais eficazes.

Na aula de hoje falaremos sobre:

- Conceito de *Feature Engineering*
- Feature Engineering na Análise Exploratória de Dados
- Algumas formas de Feature Engineering
    - Conhecimento de Negócio
    - Transformações para Distribuições Específicas
    - Discretização
    - Atributos Polinomiais
    - Séries Temporais
- Feature Engineering no Pipeline

## **1. Conceito de Feature Engineering**

A engenharia de atributos é, por definição, qualquer transformação aplicada aos dados brutos, levando-os a um patamar onde sua nova representação permita a extração de conhecimento e a sua modelagem, muitas vezes melhorando o desempenho do modelo em dados nunca vistos. Essencialmente, transformar dados brutos em atributos que melhor representam o conhecimento.

<img src="https://image.slidesharecdn.com/featureengineering-171206213206/95/feature-engineering-6-638.jpg?cb=1512596023">

Veremos que a engenharia de atributos também permitir a utilização de modelos mais simples, o que sempre ajuda do ponto de vista de custo de disponibilização em produção, desempenho, escalabilidade e também na interpretabilidade dos resultados.

Em nossas aulas anteriores, nós já fizemos alguns exemplos bem simples de engenharia de atributos, como por exemplo, a codificação das variáveis categóricas (*LabelEncoding*, *OneHotEncoding*) e também o escalonamento dos atributos (*MinMaxScaler*, *StandardScaler*, *RobustScaler*). Essas transformações são processamentos feitos aos dados para permitir o treinamento dos modelos e reduzir o viés da amostragem dos dados, mas podem ser consideradas como ferramentas de engenharia de atributos. Vejamos algumas aplicações simples do feature engineering.

## **2. Feature Engineering na Análise Exploratória de Dados**

As transformações de variáveis podem ser utilizadas durante a etapa de Análise Exploratória de Dados para gerar conhecimento a respeito dos dados. Podemos deduzir ou ter uma aproximação da função geradora dos dados observando características do seu comportamento, por meio das transformações. Vejamos um exemplo, proveniente da área de Engenharia das Reações Químicas. O problema em questão é a medida da velocidade de reação química em diversas temperaturas, passo fundamental para se realizar o dimensionamento de um reator químico.

Nesse exemplo prático, vamos estudar a velocidade de decomposição do cloreto de benzeno diazônio, um composto bem utilizado na produção de alguns tipos de corantes. No entanto, ele é bem suscetível à decomposição e sua reação é violenta, podendo produzir explosões, com extrema liberação de energia. Por isso, é muito importante conhecer o modelo que descreve a velocidade de reação em função de temperatura, de modo que se pode projetar controles eficientes de temperatura para a utilização desse componente químico.

## **3. Algumas Formas de Feature Engineering**

Verificamos que podemos estimar a função geradora ou então a relação entre duas variáveis aplicando e modificando as transformações. Nota-se também que a engenharia de atributos é um passo muito dependente do conhecimento específico do negócio - nesse caso, da engenharia química. No entanto, existem algumas técnicas que podem ser aplicadas para fazer o que chamamos de *Automatic Feature Engineering*.

A engenharia automática de atributos pode servir para realizar testes rápidos e, como o próprio nome diz, automatizados, ajudando o modelador a ter mais informações a respeito da base de dados e como se comportam as relações entre as variáveis. A primeira forma de engenharia de atributos, que já vimos, depende bastante do conhecimento de negócios e pode ser aplicada de forma direta, realizando-se cálculos com as colunas do dataset. Vejamos alguns exemplos.

### **3.1. Conhecimento de Negócio**

Nesta parte, além de cálculos diretos, podem ser realizados cálculos matemáticos como agrupamentos por categoria e diferenças em relação à uma média. Geralmente, esse tipo de transformação tende a ser bem dependente de relações já conhecidas pelo negócio, ou proveniente de discussões com o cliente, especialistas da área e pessoas mais sênior. Outros exemplos além do apresentado:

- cálculo do IMC para área médica;
- cálculo do PIB *per capita*, ou qualquer outra estatística *per capita* para estudos econômicos ou sociológicos
- cálculo do consumo específico de energia (energia consumida por kg de produto)
- cálculo de preços unitários (\$ por m², por L, por kg)
- cálculo de indicadores específicos da área de negócios
    - área financeira: TIR, VPL

### **3.2. Transformações para Distribuições Normais**

Uma das hipóteses de muitos algoritmos é a distribuição normal dos atributos. Além disso, sabe-se que modelos possuem maior desempenho quando a distribuição dos atributos se aproxima de uma normal, visto que o algoritmo consegue visualizar exemplos nos dois extremos da distribuição e assim, possui uma generalização melhor. Assim, uma possível transformação para as variáveis é o **mapeamento das mesmas para uma distribuição normal**. Dentre essas transformações, podemos citar a [QuantileTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html#sklearn.preprocessing.QuantileTransformer), a transformação de [BoxCox](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html#sklearn.preprocessing.PowerTransformer) e a transformação de [Yeo-Johnson](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PowerTransformer.html#sklearn.preprocessing.PowerTransformer).

### **3.3. Discretização e Agrupamento**

Muitas vezes, pode ocorrer do alvo possuir uma variância que não pode ser explicada totalmente pelos valores individuais dos atributos, somente. Além disso, pode acontecer de alguns atributos possuirem poucos exemplares em determinadas regiões do espaço, de forma que seria muito difícil do modelo compreender os padrões existentes nesse casos. Para alguns atributos, mesmo que ela seja contínua em sua natureza (salário de uma empresa, por exemplo), pode ser que seja interessante analisar a influência de patamares de valores, *como se ela fosse uma variável categórica*. 

### **3.4. Atributos Polinomiais**

É uma forma de adicionar complexidade num modelo, sem modificar o modelo em si, simplesmente por considerar atributos polinomiais e não lineares. Por exemplo, usando a classe `PolynomialFeatures` podemos criar mapeamentos para espaços de atributos de maior dimensão. Se usarmos utilizando um grau 2, conseguimos mapear de um espaço bidimensional para as seguintes dimensões.

$$(X_1, X_2) = (1, X_1, X_2, X_1^2, X_1 X_2, X_2^2)$$

Com isso conseguimos capturar relações não-lineares de ordem quadrática, e também conseguimos capturar as relações de interação linear dos atributos. 

**[CUIDADO]:** Devemos lembrar que apesar de ser bem útil para capturar relações não lineares, a classe `PolynomialFeatures` está **adicionando complexidade** ao modelo. É sempre bom monitorar o grau de *overfitting* do modelo, utilizando técnicas de seleção de atributos, regularização e validação cruzada.

### **3.5. Atributos Temporais**

Pode ser do interesse do analista / modelador, enxergar a influência de atributos de caráter temporal no problema. 

Por exemplo, um determinado alvo do meu modelo pode ter valores significativamente diferentes se formos considerar diferentes dias do mês ou mesmo da semana?

Desta forma, é comum, quando se trabalhar com dados que possuem características temporais, fazermos esse tipo de transformação. Vejamos:

<hr>

**Exercício 1** Teste o efeito das transformações quantílicas, Box-Cox e Yeo-Johnson no dataset `garments_worker_productivity`. Utilize apenas os atributos numéricos. Para modelagem, teste os algoritmos de Regressão Ridge e Lasso.

<hr>

**Exercício 2.** No mesmo dataset, teste a aplicação de atributos polinomiais. Teste os graus 2 e 3. Utilize apenas os atributos numéricos. Teste os algoritmos de Regressão Ridge e Lasso.